# https://github.com/kernoelpanic/smartenv

# Smart Contract Crash Course

This is an introduction to Smart Contracts based on Ethereum. Versions of this crash course have been part of:
* The [Cryptocurrencies lecture](https://tiss.tuwien.ac.at/course/courseDetails.xhtml?dswid=1923&dsrid=980&courseNr=192065&semester=2018W) at TU Wien.
* The lecture on [Advanced Topics in Internet Computing & Software Technologies](https://ufind.univie.ac.at/de/course.html?lv=052520&semester=2020W) at Uni Wien. 

## Informal introduction to Smart Contracts

* Concept/idea of smart contracts dates back to the 90s 
* Original definition is attributed to [Nick Szabo 1994](https://web.archive.org/web/20160323035617/http://szabo.best.vwh.net/smart.contracts.html):
> “A smart contract is a **computerized transaction
protocol that executes the terms of a contract**. The
general objectives are to satisfy common contractual
conditions (such as payment terms, liens, confidentiality,
and even enforcement), **minimize exceptions both
malicious and accidental**, and **minimize the need for
trusted intermediaries**. Related economic goals include
lowering fraud loss, arbitrations and enforcement costs,
and other transaction costs.”

### Let's envision such a platform, what are the goals and challenges?

* **High Level Goals:**
    + Decentralized smart contract platform
    + Verify the correct execution of each others code
        - same code was executed
        - the result/output is correct i.e., same inputs leads to same output

* **Basic Requirements:**
    + Set of nodes also called participants
    + Network connection (path to every node)
    + Code (boradcast and store on every node)
    + Input data (send to every node)

* **What can possibly go wrong? A non exhausitve list of ideas:**
    + Remote code execution vulnerability per design
        - Software security aspects
        - Sandbox/VM required
        - In Ethereuem: 
            - EVM (Ethereuem Virtual Machine)
            - no disk I/O
            - no network connection
            - only update to state variables
    + Halting problem
        - Need to limit the number of execution steps
        - In Ethereuem:
            - Pay *Gas* for every execution step
    + Correctness of the deployed code 
        - Software security aspects ...
        - In Etheruem:
            - Fail => potentialy lose funds 
    + Network attacks (MitM, eclipse attacks, DDoS, BGP attacks, ...)
        - Network security aspects, cryptography aspects, ...
        - Ensure integrity and authenticity of sent transactions
        - In Etheruem:
            - Transactions contain cryptographic signature of sender
    + Need to agree on total order of executions => consensus required
        - Distributed systems aspects, cryptography aspects, economic aspects, ...
        - In Etheruem:
            - Currently Nakamoto Consensus 
    + Privacy attacks
        - Prevent that every purchase can be traced to the involved parties.
        - Privacy enhancing technology aspects, regulator and legal aspects, ...
        - Mixing, Zero-knowledge proofs, ...

## Ethereum

Ethereum is a smart contract platform, there are also others (e.g., [Cardano](https://www.cardano.org/en/home/),[hyperleder](https://www.hyperledger.org/),[rootstock](https://www.rootstock.com/),[quorum](https://github.com/jpmorganchase/quorum), ...) but Ethereum is currently the most common. Therefore the tutorial as well as the exercises will focus on Ethereum style smart contracts. 

**Note:** A lot will (or is supposed to) change in future ethereum versions i.e., [ETH2.0](https://docs.ethhub.io/ethereum-roadmap/ethereum-2.0/eth-2.0-phases/). We focus currently on what is running at the moment i.e., ETH1.x



### Hard Facts

* Ethereum is a Cryptocurrency and smart contract platform proposed 2013 by Vitalik Buterin and Gavin Wood
* Currency symbol: ETH
* Currency name: **ether**
    + smallest unit: **wei**
    + 1 ether = 10^18 wei
    + Beware: All currency units are per default in wei!
* Technically Ethereum EVM is not Turing Complete because of Gas limits *but it doesn’t matter anyway* [1](https://media.consensys.net/ethereum-isnt-turing-complete-and-it-doesn-t-matter-anyway-625061294d3c) [2](https://www.youtube.com/watch?v=cGFOKTm_8zk)
* Solidity high-level language, syntax comparable to JavaScript
    + compiles to EVM byte code 
    + executed in EVM (Ethereum Virtual Machine)

### Overview

Etheruem in [one picture](https://github.com/kernoelpanic/BlockchainIllustrations/tree/master/Ethereum) (slightly outdated but still quite close):

<img src="https://raw.githubusercontent.com/kernoelpanic/BlockchainIllustrations/master/Ethereum/EthBlockchain5_bg.png" width=1024 height=768>

### Account model
In contrast to other Cryptocurrencies like Bitcoin (UTXO), Ethereum has the concept of **accounts** i.e., state which holds the current balance of all addresses. 

There are two types of accounts:
* **Externally controlled** accounts (aka. normal accounts)
    + Has an associated private/public key pair, like in Bitcoin
        - Address consists of the last `20` bytes of public key
        - `${Keccak­256( pk ) ):96:255}`
    + Controlled by their private keys with the respective wallet/client 
* **Contract accounts**
    + Has associated code, which is immutable after initialization (`codeHash`)
    + Has associated state (`storageRoot`)
    + Has **no** associated private/public key pair
    + Entirely controlled by its code 
    + Address is deterministically generated based on
        - address of sender and how many transactions/contract creations made so far i.e., nonce
        - or deployment code of contract (create2 opcode)
    
Moreover, each account (external and contract) has:
* `address`: 20 byte hex string, also immutable
* `balance`: Scalar value equal to the number of Wei owned by this account
* `nonce`: Scalar value equal to the number of transactions sent from this external account or, number of contract-creations made by this contract. This value is for replay protection.

### Terminology and important references 

#### Ethereum(s)
There are several Ethereum(s) (due to community disagreement about the DAO Hack), but those are (still) technically more or less the same, dispite their blockchains differ since the DAO Hack : 
* [Ethereum](https://www.ethereum.org/)
    - we will refer to this Ethereum for the rest of the lecture
    - [wiki](https://eth.wiki/en/home)
    - [ethhub](https://docs.ethhub.io/)
    - [yellow paper](https://ethereum.github.io/yellowpaper/paper.pdf)
    - [yellow paper github](https://github.com/ethereum/yellowpaper) (really the current version)
    - ~~[white paper](https://eth.wiki/en/white-Paper) (more decent introduction)~~
* [Ethereum Classic](https://ethereumclassic.github.io/)

#### Versions / Roadmap
The major versions of the Ethereum specification have dedicated names like: *Frontier*, *Homestead*, *Metropolis*, *Byzantium*, *Constantinople*, *Petersburg*, *Istanbul*, *Muir Glacier*, *Berlin*, *ETH 2.0*, ... 

#### Clients
There are also several Ethereum [clients](https://ethereum.org/en/developers/docs/nodes-and-clients/#clients) e.g.,:
* [geth](https://github.com/ethereum/go-ethereum)
    - [wiki](https://geth.ethereum.org/docs/getting-started)
    - written in go
    - we will use this client in the project/exercise
* ~~[parity](https://github.com/paritytech/parity-ethereum)~~ [openethereum](https://github.com/openethereum/openethereum)
    - written in rust
* ~~[cpp-ethereum](https://github.com/ethereum/aleth/wiki)~~
    - written in cpp
* ~~[pyethapp](https://github.com/ethereum/pyethapp)~~
    - written in python
    - mostly for dev. purposes

#### Mainnet and Testnets
There are several Ethereum [networks](https://ethereum.org/en/developers/docs/networks/) / blockchains.

* Mainnet
    - web blockchain explorers:
        - [Etherscan](https://etherscan.io/)
        - [Etherchain](https://www.etherchain.org/)
* Public Testnets e.g.,:
    - *Ropsten* 
        - PoW, should be compatiple with all clients that also work on main chain
        - [Etherscan Ropsten explorer](https://ropsten.etherscan.io/)
    - *Morden*
        - Old public PoW testnet, now Ethereum classic testnet
    - *Rinkeby*
        - Public PoA geth compatible testnet
    - *Kovan*
        - Public PoA parity compatible testnet 
    - *Görli*
        - Cross-client PoA
    
* Private or development testnets. 
    - Development testnet provided by [ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)
    - Proof-of-Authority (PoA) network provided by `geth` through *clique*.
    - ...

For the challenges we will use a private `geth` PoA network to which you will connect your nodes. 

## Connect and interface with `geth`or `ganache` using web3py

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)

Most commands issued via `web3py` are the same when connected to `ganache-cli`, or a "real" `geth` or `openethereum` node. BUT there might also be subtle differences between command when attached to `geth`, when working via `web3.py` or when working with `web3.js` or with other clients like `openethereum`. 

* [ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)
    - [API](https://github.com/trufflesuite/ganache-cli#implemented-methods)
    
* [geth console](https://geth.ethereum.org/docs/interface/javascript-console) 
    - has a link to the supported API via the geth console, which should be the "full" `web3.js` API.
    - [geth cli options](https://geth.ethereum.org/docs/interface/command-line-options) only needed when running geth from the command line

### Connect to  to `ganache-cli` development testnet via RPC / HTTP
Import the web3 libarary and connect to a running `geth`,`openethereum` or `ganache-cli` node via `HTTPProvider`


In [3]:
import web3

w3_ganache = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.2:8545"))
# check if connection was successful
assert w3_ganache.isConnected()

If connected to `geth`,`openethereuem` or as in our case to `ganache` you can execute serveral commands with the client to interact with the blockchain the client is sychronizing to. 

First it is always good the check to which node and blockchain the client is connected. In our case `ganache` starts a development chain with a random ID. The network ID provides information on which network the connected node resides.
The Ethereuem main network has id `1`.
Public test networks or private custom test networks should have different network IDs.
For a list of network IDs see this list:
* https://ethereum.stackexchange.com/questions/17051/how-to-select-a-network-id-or-is-there-a-list-of-network-ids

**Note:** Ethereuem and Ethereuem Classice have the same network ID! To avoid replay attacks of transactions the `CHAIN_ID` was introduced and is used in transactions (encoded into the signature creation, see below). 
* https://github.com/ethereum/EIPs/blob/master/EIPS/eip-155.md

In [7]:
# display the client version of the node we are connected to
w3_ganache.clientVersion

'EthereumJS TestRPC/v2.13.2/ethereum-js'

In [6]:
# display network ID of client you are connected to
# In case of ganache-cli this is a random ID
w3_ganache.net.version

'1623228734401'

### Connect to  to `geth` testnet via RPC / HTTP

Note that in our case `geth` is running in a private PoA chain setup.
Therefore, some configuration parameters are different. 

https://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority

For the challenge environment configuration the docker container for the geth client `Bob` is located at `172.18.0.8`.

In [12]:
import web3
from web3.middleware import geth_poa_middleware

w3_geth = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.8:8545"))
# check if connection was successful
assert w3_geth.isConnected()

w3_geth.middleware_onion.inject(geth_poa_middleware, layer=0)

In [13]:
# display the client version of the node we are connected to
w3_geth.clientVersion

'Geth/bob/v1.10.4-unstable-0e9c7d56-20210607/linux-amd64/go1.15.4'

In [14]:
# display network ID of client you are connected to
w3_geth.net.version

'1337'

When running `geth` you can also attach to it and use the JavaScript console (Read, Evaluate & Print Loop = REPL). 
Some stuff works in the geth console that does not work in `web3*`.
If for example `geth` is running on localhost start the binary again to attach to the running instance:
```
geth attach "http://localhost:8545"
```

The script [geth_attach.sh](../../edit/geth_attach.sh) helps you when attaching to a local geth instance which is either running in docker or natively. 

### Node information

From our `geth` node we can also retrieve some more node information:

In [20]:
w3_geth.geth.admin.nodeInfo()

AttributeDict({'id': '7faea3800bb283a57e76945abe385fe2852c0a9c6949b1961a0eadb6719e8113',
 'name': 'Geth/bob/v1.10.4-unstable-0e9c7d56-20210607/linux-amd64/go1.15.4',
 'enode': 'enode://42ee4a2ef17ab90f00c9339020124640c310e9f7521ef2d5ad377659f53e60214e8d624a525ed605f756acc06680cb33ea90575a3f6bda6ee53ec6d2bf3a8c70@127.0.0.1:30303',
 'enr': 'enr:-J24QKsqAZE-ne2uwGHI_eiKJP2bATuk4PiO5YFOVPfNbWnLXO7BeXO9uEHQHeF7p2jPlIlWCcrURPCdBtEQn1B4aXACg2V0aMfGhN-cwFGAgmlkgnY0gmlwhH8AAAGJc2VjcDI1NmsxoQJC7kou8Xq5DwDJM5AgEkZAwxDp91Ie8tWtN3ZZ9T5gIYRzbmFwwIN0Y3CCdl-DdWRwgnZf',
 'ip': '127.0.0.1',
 'ports': AttributeDict({'discovery': 30303, 'listener': 30303}),
 'listenAddr': '[::]:30303',
 'protocols': AttributeDict({'eth': AttributeDict({'network': 1337,
   'difficulty': 10155,
   'genesis': '0xa5f0a078d24ba19a4f4f01b77aa1d13d965b95dfbdca6ba2fa79e711d481272e',
   'config': AttributeDict({'chainId': 1337,
    'homesteadBlock': 0,
    'eip150Block': 0,
    'eip150Hash': '0x000000000000000000000000000000000000

### Network information

We now check if the node is connected to other peers e.g., other `geth` clients.

For our tutorial `ganache` is not connected to any peer since we have our own local development blockchain. Our web3.py connection is not a peer! The web3.py connection is just our interface to our client, therefore only local.

For the exercise your `geth` node will connect to our server in the private PoA testnet. Then you should see at least one peer i.e., our server. For a connection to be successful there are two requirements:
* Both peers need to have the same **network ID** 
* Both peers need to have the same **genesis.json** / genesis block

In [10]:
# number of connected peers to our peer
w3_ganache.net.peerCount

0

In [12]:
w3_geth.net.peerCount

1

### Syncing/blockchain information

For the exercise it will also be important to chech if your node is synchronized with the head of the blockchain i.e., our PoA node. 
If the node is started for the first time he will after some time start syncing. 
If syncing is *False* then you should see a `blockNumber` > 0

In [13]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3_ganache.eth.syncing

False

In [14]:
w3_geth.eth.syncing

False

In [25]:
# return current blockchain head of node
# Since ganache "simulates" blokchain,
# blocks are mined on demand.
# Therefore, no block has been mined yet
w3_ganache.eth.blockNumber

0

In [27]:
# Since our testnet is constantly creating blocks we should see a number > 0 
w3_geth.eth.blockNumber

5195

# Basic interaction

For all further interaction please continue with the respective clients used: 
* [geth](./sccc-geth.ipynb)
* [ganache-cli](./sccc-ganache-cli.ipynb)